# 1) Załadowanie bilbiotek

In [171]:
import pandas as pd
import statistics

In [172]:
#Read raw data
sep = '\t'
df_raw = pd.read_csv('TitanicMess.tsv', sep=sep)

# 2) Wczytanie i weryfikacja danych

#### Na początku, po wczytaniu df, sprawdzam czy wszystko wczytało się poprawnie i jak wygląda struktura dokumentu. W tym celu wywołuje funkcję head i tail.

In [173]:
display(df_raw.head(5))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic


In [174]:
display(df_raw.tail(5))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30,B42,S,Titanic
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,"23,45",NaN,S,Titanic
889,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30,C148,C,Titanic
890,891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,"7,75",NaN,Q,Titanic
891,1000,1,1,Mr. Frederick Maxfield Hoyt,male,38,1,0,19943,90,C93,S,Titanic


### Poniżej widać, że dość problematycznymi kolumnami są Ticket oraz Name. 
#### Wynika to po pierwsze niekonsekwencji we wprowadzaniu danych, a po drugie z ich duplikacji (kilka osób może mieć ten sam numer biletu). 
#### Można jednak założyć, że do analizy nie będą potrzebne takie dane jak nazwiska i numery biletów. Zdecydowałem się więc je usunąć. 
<br>
 (bilety można by naprawić za pomocą regexu jednak ich wartość dla analizy jest żadna, z tego powodu ekonomiczniej będzie je usunąć)

In [175]:
df_raw.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic


In [176]:
#Wszystkie wartości kolumny ship
print(df_raw['ship'].unique())


['Titanic']


#### 1) jak widać powyżej, kolumna ship również nie przyda się w analizie ponieważ zawiera tylko jedną wartość
#### 2) podobnie jest z numerami kabin. Można je częściowo uzupełnić po numerach biletu, jednak liczba braku danych w tej kategorii i nikła wartość w analizie sugeruje ich usunięcie, co też robie.

In [177]:
#Budowanie DF bez zbędnych kolumn
df_raw = df_raw[['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

# 3) Wstępne czyszczenie danych

#### Następnym krokiem jest usunięcie rekordów zduplikowanych i rekordów zawierających braki danych

##### oczywiście z brakiem danych można poradzić sobie na wiele różnych sposobów - powinno się dobrać odpowiednią metodę pod konkretny cel analizy. Ja zdecydowałem się je usunąć. 


* Braki danych usuwam za pomocą napisanej funkcji. Wcześniejsze usunięcie kolumn nieistotnych do analizy gwarantuje mi w tym kroku zachowanie znacznie większej liczby cennych danych

* Zduplikowane wiersze są usuwane po PassangerId. Późniejszą weryfikacja upewniam się, że ten zabieg wystarczy.

In [178]:
#funkcja usuwająca braki danych i sprawdzająca czy istnieją duplikaty. Jeśli istnieją, to funkcja je usuwa. Zwracana jest wyczyszczona ramka.

def delete_NA_and_duplicates(df):
    
    #delete NA
    raw_len = len(df)
    df = df.dropna()
    print("Deleted NA rows: {}".format(raw_len - len(df)))
    
    #delete duplicates
    dup = len(df[df['PassengerId'].duplicated()])
    if dup > 0:
        df = df.drop_duplicates(subset='PassengerId', keep="last")
        print("Deleted Duplicates rows by PassengerId: {}".format(dup))
       
        return df
    else:
        return df
    
    
df = delete_NA_and_duplicates(df_raw)

Deleted NA rows: 175
Deleted Duplicates rows by PassengerId: 4


In [179]:
#Sprawdzenie duplikatów
df[df.duplicated()]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked


# 4) Weryfikacja wyczyszczonych danych

#### Na wyczyszczonych wstępnie danych używam funkcji info, aby uzyskać podstawowe informacje o tabeli
* Jak widac, kolumna Age jest w tym przypadku jest typu object. Są to wartości, które powinny być zapisane jako Int, co sugeruje, że należy się jej przyjrzeć.

In [180]:
#Basic info about df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 713 entries, 0 to 891
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  713 non-null    int64 
 1   Survived     713 non-null    int64 
 2   Pclass       713 non-null    int64 
 3   Sex          713 non-null    object
 4   Age          713 non-null    object
 5   SibSp        713 non-null    int64 
 6   Parch        713 non-null    int64 
 7   Fare         713 non-null    object
 8   Embarked     713 non-null    object
dtypes: int64(5), object(4)
memory usage: 55.7+ KB


#### Sprawdzam jak wygląda kolumna Age za pomocą funkcji count_values()

In [181]:
df.Survived.unique()

array([0, 1], dtype=int64)

In [182]:
#Fix Age col
testAge = df['Age'].value_counts().to_dict()

In [183]:
display(testAge)

{'24': 30,
 '22': 27,
 '18': 26,
 '19': 25,
 '28': 24,
 '21': 24,
 '30': 24,
 '36': 22,
 '25': 22,
 '29': 20,
 '27': 18,
 '26': 18,
 '32': 18,
 '35': 18,
 '16': 17,
 '31': 17,
 '23': 15,
 '34': 15,
 '33': 15,
 '20': 15,
 '39': 13,
 '42': 13,
 '40': 13,
 '17': 13,
 '45': 12,
 '38': 11,
 '50': 10,
 '4': 10,
 '2': 10,
 '47': 9,
 '48': 9,
 '44': 9,
 '54': 8,
 '9': 8,
 '51': 7,
 '1': 7,
 '41': 6,
 '37': 6,
 '14': 6,
 '49': 6,
 '52': 6,
 '3': 6,
 '15': 5,
 '58': 5,
 '11': 4,
 '60': 4,
 '8': 4,
 '56': 4,
 '5': 4,
 '43': 4,
 '61': 3,
 '46': 3,
 '65': 3,
 '7': 3,
 '6': 3,
 '62': 3,
 '63': 2,
 '57': 2,
 '32,5': 2,
 '70': 2,
 '10': 2,
 '45,5': 2,
 '0,75': 2,
 '59': 2,
 '28,5': 2,
 '0,83': 2,
 '40,5': 2,
 '71': 2,
 '64': 2,
 '13': 2,
 '30,5': 2,
 '55': 2,
 '0,67': 1,
 '55,5': 1,
 '14,5': 1,
 '36,5': 1,
 '74': 1,
 '70,5': 1,
 '23,5': 1,
 '0,42': 1,
 '-12': 1,
 '20,5': 1,
 '80': 1,
 '66': 1,
 '.9': 1,
 '12': 1,
 '53': 1,
 '24,5': 1,
 '0,92': 1,
 '4435': 1,
 '34,5': 1,
 '-3': 1,
 '250': 1}

#### W kolumnie Age mamy dane zawierające wartości dziesiętne. Nie ma możliwości sprawdzenia ich poprawności, dalego wszystkie dane o wartości niecałkowitej zostaną usunięte.

In [184]:
df = df[df.Age.apply(lambda x: x.isnumeric())]

In [185]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22,1,0,"7,25",S
1,2,1,1,female,38,1,0,"71,2833",C
2,3,1,3,female,26,0,0,"7,925",S
3,4,1,1,female,35,1,0,"53,1",S
4,5,0,3,male,35,0,0,"8,05",S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27,0,0,13,S
887,888,1,1,female,19,0,0,30,S
889,890,1,1,male,26,0,0,30,C
890,891,0,3,male,32,0,0,"7,75",Q


# 5) Ponowna Weryfikacja

* za pomocą wartości unikalnych mogę szybko przejrzeć poprawność powstałych danych

In [186]:
def createString(string):
    
    try:
        print(10*"-" + string + 10*"-")
    except:
        print('Smth wrong')
        
def valueCounts(df):
    
    num_of_cols = len(df.columns)
    
    for i, j in enumerate(df.columns):
        print(j)
        createString("Unikalne {}".format(j))
        print(df[j].value_counts())
    

In [187]:
valueCounts(df)

PassengerId
----------Unikalne PassengerId----------
1000    1
294     1
311     1
310     1
309     1
       ..
600     1
598     1
596     1
595     1
1       1
Name: PassengerId, Length: 685, dtype: int64
Survived
----------Unikalne Survived----------
0    406
1    279
Name: Survived, dtype: int64
Pclass
----------Unikalne Pclass----------
3    337
1    181
2    167
Name: Pclass, dtype: int64
Sex
----------Unikalne Sex----------
male       431
female     251
femmale      1
malef        1
mal          1
Name: Sex, dtype: int64
Age
----------Unikalne Age----------
24      30
22      27
18      26
19      25
28      24
        ..
66       1
80       1
53       1
4435     1
74       1
Name: Age, Length: 72, dtype: int64
SibSp
----------Unikalne SibSp----------
0    450
1    177
2     23
4     18
3     12
5      5
Name: SibSp, dtype: int64
Parch
----------Unikalne Parch----------
0    503
1    105
2     64
3      5
5      4
4      4
Name: Parch, dtype: int64
Fare
----------Unikalne Fare-

### Powyższe podsumowanie wskazuje jeszcze kilka błędów które należy naprawić ręcznie

##### 1) Kolumna Sex ma błędy w nazewnictwie. Przez co w trzech wierszach jest błędnie napisane female/male

##### 2) Kolumna Age zawiera nierzeczywistą wartość - ponad 4000 lat. Należy usunąć ten wiersz.

##### 3) Kolumna Embraked zawiera analogiczny jak w przypadku Sex błąd. Należy go ręcznie naprawić.

# 6) Ręczne dopracowanie danych

### Zmiana błędnych wartości w Sex

In [188]:
df = df.copy()
df['Sex'] = df['Sex'].replace({'malef': 'male', 'mal': 'male', 'femmale':'female'})

In [189]:
#Walidacja danych Sex
print(df['Sex'].unique())

['male' 'female']


### Usuwanie błędnych wartości w kolumnie Age


In [190]:
#Zmiana typu Age ze string na INT oraz zmiana Survived z int na Boolean
df = df.astype({"Age": int, "Survived":bool})

In [191]:
#Sprawdzam wiek uczestników powyżej 90 lat. Usuwam uszkodzone dane
print(df[df['Age'] > 90])
df = df[df['Age'] < 90]

In [192]:
#Walidacja danych Age
print("MAX AGE: {}".format(df['Age'].max()))
print("MIN AGE: {}".format(df['Age'].min()))

MAX AGE: 80
MIN AGE: 1


### Zmiana błędnych wartości w Embraked

In [193]:
df = df.copy()
df['Embarked'] = df['Embarked'].replace({'So':'S', 'Qe':'Q'})


In [194]:
#Walidacja danych Embraked
df['Embarked'].unique()

array(['S', 'C', 'Q'], dtype=object)

# 7) Zapis oczyszconego df do pliku

In [196]:
df.to_csv('TitanicCleaned.tsv', sep = sep, index = False)

# WNIOSKI

Plik wymagał usunięcia duplikatów, usunięcia zbędnych kolumn, usunięcia wierszy z wartościami brakującymi, ujednolicenia niektórych wartości.

Uzupełnienie brakujących danych było w przypadku niektórych danych nieopłacalne lub niemożliwe. Przykładowo kabiny zawierały ogromny odsetek danych brakujących. Nie były także możliwe do uzupełnienia (można było próbować mapować numery kabin po numerze biletów, ale tam również brakowało danych + uzyskane dane nie byłyby wystarczająco "pewne"). Dzięki usunięciu zbędnych kolumn, późniejsze czyszczenie tabeli z wartości NaN pozwoliło zachować ponad 680 wierszy, gdy z zachowanymi kolumnami byłoby ich około 170. Uznałem więc, że lepiej jest zachować ponad 4-krotnie więcej danych poprawnych i uzupełnionych, bez nieistotnych kolumn, niż zachowywać kolumny i drastycznie zmniejszyć liczbę obserwacji.

Chciałbym także zaznaczyć, że o ile skalowalność w tego typu skryptach jest jak najbardziej możliwa i zalecana (i starałem się ją zachować, używając w większości funkcji pandas, działających znacznie szybciej niż np. pętle for i możliwych do zastosowania w każdym innym DF), to używanie tego typu skryptów dla innych danych wejściowych, niż tych do których zostały napisane, jest często niemożliwe lub nieopłacalne, zwłaszcza jeśli chcemy by dany skrypt działał automatycznie. Widać to choćby na przykładzie literówek w kolumnie Sex. Programista musiałby przewidzieć możliwe błędy, bądź napisać algorytm je wykrywający, by nie tracić zbyt dużej ilości danych. 

Przykładowo, gdybyśmy chcieli teraz załadować zamiast pliku titanic, jakiś inny DF np. z danymi technicznymi samochodów osobowych, musielibyśmy przewidzieć programowo, które kolumny są mniej istotne, a które bardziej, jakie błędy mogą wystąpić i do jakiej analizy chcemy przeznaczyć plik wynikowy. Oznacza to, że większość przykładów będzie indywudalna i ciężko będzie je zautomatyzować jednym skryptem - jak czyszczenie danych -, by dane te były jak najbardziej przydatne do późniejszej analizy. 
